In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
import copy
from sklearn.utils import shuffle

In [36]:
lda_model = models.LdaModel.load('models/best_lda_topic_model.gensim')

In [37]:
dictionary = gensim.corpora.Dictionary.load('models/best_lda_topic_model_dictionary.gensim')

In [112]:
corpus = pd.read_csv('learners_contribution.csv')

In [114]:
corpus

,Unnamed: 0,contribution_id,user_id,mandate,type,order,date_time,full_text,description,clean_text
8,8,9,14,mandate_1,discussion,1,"10 September 2018, 3:17 PM",Whatsapp has more number of users than telegra...,Whatsapp has more number of users than telegra...,whatsapp ha number user telegram even though t...
13,13,14,prof,mandate_1,discussion,2,"10 September 2018, 4:36 PM","Providing a helpful article is fine, but a dis...","Providing a helpful article is fine, but a dis...",providing helpful article fine discussion foru...
21,21,22,52,mandate_2,discussion,1,"6 November 2018, 3:17 PM","centrality measure In the class, construction ...","centrality measure In the class, construction ...",centrality measure class construction hospital...
9,9,10,prof,mandate_1,discussion,2,"10 September 2018, 4:35 PM",Question needs more details. What are the more...,Question needs more details. What are the more...,question need details feature telegram ha what...
18,18,19,55,mandate_1,seminar,1,"17 September 2018, 3:15 PM",Power Law Distribution Janardhana Anipireddy...,Power Law Distribution Janardhana Anipireddy...,power law distribution janardhana anipireddy p...
19,19,20,30,mandate_1,seminar,1,"11 September 2018, 8:08 PM",Se rz (e (= Mate lt aes Eigenvector Centralit...,Se rz (e (= Mate lt aes Eigenvector Centralit...,se rz mate lt aes eigenvector centrality semin...
22,22,23,m55,mandate_2,discussion,1,"30 October 2018, 4:22 PM",Contrast between Centrality and Vitality Centr...,Contrast between Centrality and Vitality Centr...,contrast centrality vitality centrality measur...
7,7,8,prof,mandate_1,reply/feedback,2,"10 September 2018, 4:39 PM",Technical content of the essay can be greatly ...,Technical content of the essay can be greatly ...,technical content essay greatly enhanced essay...
2,2,3,37,mandate_1,essay,1,"18 September 2018, 12:48 PM",R.Lakshmi Priya Prof. Srinath Srinivasa 18 Se...,R.Lakshmi Priya Prof. Srinath Srinivasa 18 Se...,lakshmi priya prof srinath srinivasa september...
20,20,21,m33,mandate_1,seminar,1,"11 September 2018, 5:32 PM","Unsupervised Learning, Exploratory Analysis an...","Unsupervised Learning, Exploratory Analysis an...",unsupervised learning exploratory analysis ran...


In [5]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [6]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [38]:
lcorp = pd.read_csv('learners_contribution.csv')

In [39]:
processed_docs = tokenize(lcorp['clean_text'][2])
processed_docs = [w for w in processed_docs if len(w) > 2]
bow_vector = dictionary.doc2bow(processed_docs)

In [40]:
lda_model[bow_vector]

[(5, 0.03309608), (6, 0.96089876)]

In [41]:
processed_docs = get_tokens(lcorp['clean_text'])

In [42]:
l = []
for i in processed_docs:
    bow_vector = dictionary.doc2bow(i)
    r = [0]*8
    for j in lda_model[bow_vector]:
        r[j[0]] = j[1]
    l.append(r)

In [43]:
df = pd.DataFrame(l)

In [51]:
df = pd.concat([lcorp['contribution_id'],df],axis=1)

In [53]:
df.to_csv('models/prob_dist.csv')